In [ ]:
import re
import sys
import string
import itertools
from collections import Counter

import pandas as pd
import numpy as np
import kagglehub
from pathlib import Path

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import spacy


from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from tqdm.auto import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import seaborn as sns

import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors # implements word vectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

import matplotlib.cm as cm
import datetime


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nolan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nolan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [81]:
# 1. Environment diagnostics
if 'google.colab' in sys.modules:
    print("➡️ Running in Colab")
else:
    print("➡️ Running in local Python / VS Code")

# 2. Downloading the dataset via kagglehub
ds_path = kagglehub.dataset_download("arhamrumi/amazon-product-reviews")
path = Path(ds_path)

print("Path to dataset files:", ds_path)  

print("Dataset root:", ds_path)

➡️ Running in local Python / VS Code
Path to dataset files: C:\Users\Nolan\.cache\kagglehub\datasets\arhamrumi\amazon-product-reviews\versions\1
Dataset root: C:\Users\Nolan\.cache\kagglehub\datasets\arhamrumi\amazon-product-reviews\versions\1


In [82]:
df = pd.read_csv(path/'Reviews.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Набір даних містить такі ознаки для кожного відгуку:

**Id** — ідентифікатор запису в наборі даних;  
**ProductId** — унікальний ідентифікатор товару;  
**UserId** — унікальний ідентифікатор користувача;  
**ProfileName** — ім'я профілю користувача;  
**HelpfulnessNumerato**r — кількість користувачів, які вважали огляд корисним;  
**HelpfullnessDenominator** — кількість користувачів, які вказали, чи вважають вони відгук корисним чи ні;  
**Score** — оцінка від 1 до 5;  
**Time** — позначка часу перегляду відгуку;  
**Summary** — короткий зміст відгуку;  
**Text** — текст відгуку.

In [83]:
print(f"Count of uniq: {df.nunique()}", '\n\n')
print(df.info())

Count of uniq: Id                        568454
ProductId                  74258
UserId                    256059
ProfileName               218415
HelpfulnessNumerator         231
HelpfulnessDenominator       234
Score                          5
Time                        3168
Summary                   295742
Text                      393579
dtype: int64 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568428 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64

In [84]:
before = df.shape[0]
df = df.loc[df['Score'] != 3]
after = df.shape[0]
print("Removed:", before - after, "Remaining:", after)
df.shape


Removed: 42640 Remaining: 525814


(525814, 10)

In [85]:
'''Створимо цільову змінну sentiment для розв’язання задачі бінарної класифікації. 
Наразі проігноруємо записи з нейтральним сентиментом, тобто ті, які мають рейтинг 3'''
df['sentiment'] = [1 if score in [4, 5] else 0 for score in df['Score']]

In [86]:
df.drop(columns=['Id'], inplace=True)

In [87]:
df.duplicated().sum()

np.int64(255)

In [88]:
df.drop_duplicates().reset_index(drop=True)

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,sentiment
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1
...,...,...,...,...,...,...,...,...,...,...
525554,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...,1
525555,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...,0
525556,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o...",1
525557,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...,1


In [89]:
df_group_by_prod = df.groupby(['UserId', 'Time', 'Summary']).count().sort_values('ProductId', ascending=False)
df_group_by_prod

,,,ProductId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Text,sentiment
UserId,Time,Summary,,,,,,,
A3TVZM3ZIXG8YW,1291420800,"Filler food is empty, leaves your cat always needing more",199,199,199,199,199,199,199
A29JUMRL1US6YP,1278201600,Fantastic Food for Good Cat Health,125,125,125,125,125,125,125
A3IE3TQ79W0YOR,1280793600,Please avoid!!,42,42,42,42,42,42,42
A20P8VC55KPPCT,1310860800,Love all HappyBaby Tots!,40,40,40,40,40,40,40
AGPLJLVQ47QLT,1322870400,DOOMHAMMER approved,36,36,36,36,36,36,36
...,...,...,...,...,...,...,...,...,...
AZZV9PDNMCOZW,1329436800,Just what I needed,1,1,1,1,1,1,1
AZZVNIMTTMJH6,1268179200,pretty amazing,1,1,1,1,1,1,1
AZZY649VYAHQS,1309737600,I Have Spoken.,1,1,1,1,1,1,1


In [90]:
search_text = 'Filler food is empty, leaves your cat always needing more'
duplicates_example = df.loc[
    (df['UserId']=='A3TVZM3ZIXG8YW') &
    (df['Time']==1291420800) &
    (df['Summary']==search_text)
]

duplicates_example

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,sentiment
2941,B0002TJAZK,A3TVZM3ZIXG8YW,christopher hayes,7,11,1,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ...",0
2947,B0002TJAZK,A3TVZM3ZIXG8YW,christopher hayes,0,2,1,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ...",0
31782,B00106TG9Y,A3TVZM3ZIXG8YW,christopher hayes,2,8,1,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ...",0
52496,B003ANFMY8,A3TVZM3ZIXG8YW,christopher hayes,19,21,1,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ...",0
52501,B003ANFMY8,A3TVZM3ZIXG8YW,christopher hayes,18,24,1,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ...",0
...,...,...,...,...,...,...,...,...,...,...
499916,B009B87SAC,A3TVZM3ZIXG8YW,christopher hayes,6,14,1,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ...",0
499917,B009B87SAC,A3TVZM3ZIXG8YW,christopher hayes,6,15,1,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ...",0
514140,B003M5VM8O,A3TVZM3ZIXG8YW,christopher hayes,5,9,1,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ...",0
514690,B003MWGSKY,A3TVZM3ZIXG8YW,christopher hayes,3,8,1,1291420800,"Filler food is empty, leaves your cat always n...","This review will make me sound really stupid, ...",0


In [91]:
df = df.drop_duplicates(subset=['UserId', 'Time', 'Text'])
df.shape

(364133, 10)

In [92]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}


In [93]:
stop_words = set(stopwords.words('english')).union({'also', 'would', 'much', 'many'})


In [94]:
negations = {
    'aren',
    "aren't",
    'couldn',
    "couldn't",
    'didn',
    "didn't",
    'doesn',
    "doesn't",
    'don',
    "don't",
    'hadn',
    "hadn't",
    'hasn',
    "hasn't",
    'haven',
    "haven't",
    'isn',
    "isn't",
    'mightn',
    "mightn't",
    'mustn',
    "mustn't",
    'needn',
    "needn't",
    'no',
    'nor',
    'not',
    'shan',
    "shan't",
    'shouldn',
    "shouldn't",
    'wasn',
    "wasn't",
    'weren',
    "weren't",
    'won',
    "won't",
    'wouldn',
    "wouldn't"
}
stop_words = stop_words.difference(negations)


In [95]:
stemmer = PorterStemmer()

In [96]:
nlp = spacy.load("en_core_web_sm", disable = ['parser','ner'])
def normalize_text(raw_review):
    
    # Remove html tags
    text = re.sub("<[^>]*>", " ", raw_review) # match <> and everything in between. [^>] - match everything except >
    
    # Remove emails
    text = re.sub("\\S*@\\S*[\\s]+", " ", text) # match non-whitespace characters, @ and a whitespaces in the end
    
    # remove links
    text = re.sub("https?:\\/\\/.*?[\\s]+", " ", text) # match http, s - zero or once, //, 
                                                    # any char 0-unlimited, whitespaces in the end
        
     # Convert to lower case, split into individual words
    text = text.lower().split()
    
    # Replace contractions with their full versions
    text = [contractions.get(word) if word in contractions else word 
            for word in text]
   
    # Re-splitting for the correct stop-words extraction
    text = " ".join(text).split()    
    
    # Remove stop words
    text = [word for word in text if not word in stop_words]

    text = " ".join(text)
    
    # Remove non-letters        
    text = re.sub("[^a-zA-Z' ]", "", text) # match everything except letters and '

    # Stem words. Need to define porter stemmer above
    # text = [stemmer.stem(word) for word in text.split()]

    # Lemmatize words. Need to define lemmatizer above
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc if len(token.lemma_) > 1 ])
    
    # Remove excesive whitespaces
    text = re.sub("[\\s]+", " ", text)    
    
    # Join the words back into one string separated by space, and return the result.
    return text


In [97]:
text = '''On a quest for the perfedc1112t,,, !!!! <br />%%2%% popcorn to compliment the Whirley Pop.  
Don\\'t get older, I\\'m beginning to appreciate the more "natural" popcorn varieties, and I suppose that\\'s
 what attracted me to the Arrowhead Mills Organic Yellow Popcorn.<br /> <br />I\\'m no "organic" food expert. 
   I just wanted some good tasting popcorn.  And, I feel like that\\'s what I got.  Using the Whirley Pop, with a very small amount of oil, I\\'ve had great results.'''

print("Original text:\n", text, "\n", "#"*30, "\n", sep="")
print("Normalized text:\n", normalize_text(text), sep="")



Original text:
On a quest for the perfedc1112t,,, !!!! <br />%%2%% popcorn to compliment the Whirley Pop.  
Don\'t get older, I\'m beginning to appreciate the more "natural" popcorn varieties, and I suppose that\'s
 what attracted me to the Arrowhead Mills Organic Yellow Popcorn.<br /> <br />I\'m no "organic" food expert. 
   I just wanted some good tasting popcorn.  And, I feel like that\'s what I got.  Using the Whirley Pop, with a very small amount of oil, I\'ve had great results.
##############################

Normalized text:
quest perfedct popcorn compliment whirley pop do not get old be begin appreciate natural popcorn variety suppose that be attract arrowhead mill organic yellow popcorn be no organic food expert want good tasting popcorn and feel like that be get use whirley pop small amount oil have great result


In [98]:
# df = df.groupby('sentiment').sample(2500, random_state=42)

# df.shape


In [99]:
df['text_normalized'] = df['Text'].progress_apply(normalize_text)

100%|██████████| 364133/364133 [23:16<00:00, 260.72it/s]


In [100]:
df.head()

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,sentiment,text_normalized
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1,buy several vitality can dog food product find...
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0,product arrive label jumbo salt peanutsthe pea...
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1,confection around century light pillowy citrus...
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0,look secret ingredient robitussin believe find...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1,great taffy great price wide assortment yummy ...


In [101]:
train_idxs = df.sample(frac=0.8, random_state=42).index
test_idxs = [idx for idx in df.index if idx not in train_idxs]

X_train = df.loc[train_idxs, 'text_normalized']
y_train = df.loc[train_idxs, 'sentiment']

X_test = df.loc[test_idxs, 'text_normalized']
y_test = df.loc[test_idxs, 'sentiment']

In [102]:
vect = CountVectorizer().fit(X_train)

len(vect.vocabulary_)

148264

In [113]:
vect

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"
,analyzer,'word'


In [103]:
vect.get_feature_names_out()[:5]

array(['aa', 'aaa', 'aaaa', 'aaaaa', 'aaaaaa'], dtype=object)

In [104]:
X_train_vectorized = vect.transform(X_train)

print(X_train_vectorized.shape)
X_train_vectorized

(291306, 148264)


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 9609419 stored elements and shape (291306, 148264)>

In [114]:
model = LogisticRegression(random_state=42)
model.fit(X_train_vectorized, y_train)
proba = model.predict_proba(vect.transform(X_test))[:, 1]

print('AUC: ', roc_auc_score(y_test, proba))

c:\Users\Nolan\Documents\My dokuments\GoIT\Deep_learning\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AUC:  0.9403412285150121


In [106]:
list(vect.vocabulary_.items())[:20]

[('wow', 145862),
 ('pleasantly', 98234),
 ('surprised', 126389),
 ('product', 101848),
 ('love', 74696),
 ('eat', 39745),
 ('dry', 38845),
 ('fruit', 50953),
 ('handy', 58028),
 ('pack', 93389),
 ('great', 55626),
 ('run', 110448),
 ('treat', 134494),
 ('buying', 17480),
 ('lunchboxe', 75322),
 ('asian', 6918),
 ('pear', 95392),
 ('daughter', 32542),
 ('favorite', 45648),
 ('please', 98247)]

Для подальшого швидкого тестування різних варіантів обробки даних напишемо допоміжну функцію.

In [107]:
def get_preds(text_column, algorithm, ngrams=(1,1)):
    
    X_train = df.loc[train_idxs, text_column]
    X_test = df.loc[test_idxs, text_column]

    y_train = df.loc[train_idxs, 'sentiment']
    y_test = df.loc[test_idxs, 'sentiment']
    
    if algorithm == 'cv':
        vect = CountVectorizer(ngram_range=ngrams).fit(X_train)
    elif algorithm == 'tfidf':
        vect = TfidfVectorizer(ngram_range=ngrams).fit(X_train)
    else:
        raise ValueError('Select correct algorithm: `cv` or `tfidf`')
            
    print('Vocabulary length: ', len(vect.vocabulary_))
    
    # transform the documents in the training data to a document-term matrix

    X_train_vectorized = vect.transform(X_train)
    print('Document-term matrix shape:', X_train_vectorized.shape)
    
    model = LogisticRegression(random_state=42)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict_proba(vect.transform(X_test))[:, 1]

    print('AUC: ', roc_auc_score(y_test, predictions))


In [108]:
get_preds('text_normalized', 'cv')


Vocabulary length:  148264
Document-term matrix shape: (291306, 148264)


c:\Users\Nolan\Documents\My dokuments\GoIT\Deep_learning\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AUC:  0.9403412285150121


In [109]:
get_preds('text_normalized', 'tfidf')


Vocabulary length:  148264
Document-term matrix shape: (291306, 148264)
AUC:  0.9542780328549247


In [112]:
get_preds('text_normalized', 'cv', (1,2))

Vocabulary length:  2999090
Document-term matrix shape: (291306, 2999090)


c:\Users\Nolan\Documents\My dokuments\GoIT\Deep_learning\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AUC:  0.9631257324144284


In [111]:
get_preds('text_normalized', 'tfidf', (1,2))

Vocabulary length:  2999090
Document-term matrix shape: (291306, 2999090)
AUC:  0.9672269189863768


In [115]:
def build_corpus(data):
  corpus = []
  for sentence in data:
    word_list = sentence.split(" ")
    corpus.append(word_list)
  return corpus


In [121]:
corpus = build_corpus(df['text_normalized'])
corpus[0]


['buy',
 'several',
 'vitality',
 'can',
 'dog',
 'food',
 'product',
 'find',
 'good',
 'quality',
 'product',
 'look',
 'like',
 'stew',
 'process',
 'meat',
 'smell',
 'well',
 'labrador',
 'finicky',
 'appreciate',
 'product',
 'well',
 'most']

In [130]:
model_emb_from_scratch = word2vec.Word2Vec(corpus, vector_size=100, window=5, min_count=50, workers=4)

# saving vectors
model_emb_from_scratch.wv.save_word2vec_format('model_emb_from_scratch.bin', binary=True)

In [131]:
# Завантажуємо вектори з файлу
my_vectors = KeyedVectors.load_word2vec_format('model_emb_from_scratch.bin', binary=True)

print("Вектори успішно завантажено!")

Вектори успішно завантажено!


In [ ]:
# Спробуй щось специфічне для Amazon, наприклад 'expensive' (дорогий)
print("\nСлова, схожі на 'expensive':")
try:
    print(my_vectors.most_similar('expensive'))
except KeyError:
    pass


Слова, схожі на 'expensive':
[('costly', 0.8722464442253113), ('pricey', 0.8475409746170044), ('cheap', 0.789085865020752), ('pricy', 0.7803586721420288), ('overpriced', 0.7537978887557983), ('pricier', 0.7172571420669556), ('cost', 0.711898922920227), ('spendy', 0.6908905506134033), ('cheaper', 0.6570128798484802), ('overprice', 0.6349856853485107)]


In [134]:
class WordEmbedding: 

    def __init__(self):
        self.model = {}
        
    def convert(self, source, ipnut_file_path, output_file_path):
        '''
        Converts word embeddings from GloVe format to Word2Vec format
        '''
        if source == 'glove':
            input_file = datapath(ipnut_file_path)
            output_file = get_tmpfile(output_file_path)
            glove2word2vec(input_file, output_file)
        elif source in ['word2vec', 'fasttext', 'from_scratch']:
            pass
        else:
            raise ValueError('Possible value of source are glove, word2vec, fasttext, or from_scratch')
        
    def load(self, source, file_path):
        '''
        Loads a specified word embedding model from a file
        '''
        print(datetime.datetime.now(), 'start: loading', source)
        if source in ['glove', 'fasttext']:
            self.model[source] = gensim.models.KeyedVectors.load_word2vec_format(file_path)
        elif source in ['word2vec', 'from_scratch']:
            self.model[source] = gensim.models.KeyedVectors.load_word2vec_format(file_path, binary=True)
        else:
            raise ValueError('Possible value of source are glove, word2vec, fasttext, or from_scratch')
            
        print(datetime.datetime.now(), 'end: loading', source)
            
        return self
    
    def get_model(self, source):
        '''
        Retrieves the loaded word embedding model
        '''
        if source not in ['glove', 'word2vec', 'fasttext', 'from_scratch']:
            raise ValueError('Possible value of source are glove, word2vec, fasttext, or from_scratch')
            
        return self.model[source]
    
    def get_words(self, source, size=None):
        '''
        Retrieves a list of words from the model
        '''
        if source not in ['glove', 'word2vec', 'fasttext', 'from_scratch']:
            raise ValueError('Possible value of source are glove, word2vec, fasttext, or from_scratch')

        if size is None:
            return [w for w in self.get_model(source=source).key_to_index]
        else:
            results = []
            for i, word in enumerate(self.get_model(source=source).key_to_index):
                if i >= size:
                    break
                results.append(word)
            return results
        
        return Exception('Unexpected flow')
    
    def get_dimension(self, source):
        '''
        Retrieves the dimension of word vectors in the model
        '''
        if source not in ['glove', 'word2vec', 'fasttext', 'from_scratch']:
            raise ValueError('Possible value of source are glove, word2vec, fasttext, or from_scratch')
        
        return self.get_model(source=source).vectors[0].shape[0]
    
    def get_vectors(self, source, words=None):
        '''
        Retrieves vectors for specified words or for all words in the model
        '''
        if source not in ['glove', 'word2vec', 'fasttext', 'from_scratch']:
            raise ValueError('Possible value of source are glove, word2vec, fasttext, or from_scratch')
        
        if words is None:
            words = self.get_words(source=source)
            
        embedding = np.empty((len(words), self.get_dimension(source=source)), dtype=np.float32)
        for i, word in enumerate(words):
            embedding[i] = self.get_vector(source=source, word=word)
                
        return embedding
        
    
    def get_vector(self, source, word):
        '''
        Retrieves the vector representation of a single word
        '''
        if source not in ['glove', 'word2vec', 'fasttext', 'from_scratch']:
            raise ValueError('Possible value of source are glove, word2vec, fasttext, or from_scratch')
            
        if source not in self.model:
            raise ValueError('Did not load %s model yet' % source)
        
        try:
            return self.model[source][word]
        except KeyError as e:
            dims = self.model[source][0].shape
            vect = np.empty(dims)
            vect[:] = np.nan
            return vect
            
    def get_synonym(self, source, word, topn=5):
        '''
        Retrieves synonyms for a given word
        '''
        if source not in ['glove', 'word2vec', 'fasttext', 'from_scratch']:
            raise ValueError('Possible value of source are glove, word2vec, fasttext, or from_scratch')
            
        if source not in self.model:
            raise ValueError('Did not load %s model yet' % source)
        
        try:
            return self.model[source].most_similar(positive=word, topn=topn)
        except KeyError as e:
            raise
    
    def get_distance_between_two_words(self, source, word1, word2):
        '''
        Calculates cosine similarity between two words in the model
        '''
        if source not in ['glove', 'word2vec', 'fasttext', 'from_scratch']:
            raise ValueError('Possible value of source are glove, word2vec, fasttext, or from_scratch')
            
        if source not in self.model:
            raise ValueError('Did not load %s model yet' % source)
        
        try:
            return self.model[source].similarity(word1, word2)
        except KeyError as e:
            raise


In [141]:
word_embedding = WordEmbedding()

In [142]:
# word2vec_file_path = 'pretrained_models/GoogleNews-vectors-negative300.bin'

# fasttext_file_path = 'pretrained_models/wiki-news-300d-1M.vec'

from_scratch_file_path = 'model_emb_from_scratch.bin'

# adding absolute path for correct gensim work
# downloaded_glove_file_path = os.path.abspath('pretrained_models') + '/glove.6B.50d.txt'
# glove_file_path = os.path.abspath('pretrained_models') + '/glove.6B.50d.vec'

try:
    word_embedding.load(source='from_scratch', file_path=from_scratch_file_path)
except FileNotFoundError:
    print(f"Помилка: Файл {from_scratch_file_path} не знайдено. Перевір шлях!")

2025-12-18 15:12:03.216887 start: loading from_scratch
2025-12-18 15:12:03.248410 end: loading from_scratch


In [ ]:
# word_embedding.convert(source='glove', ipnut_file_path=downloaded_glove_file_path, output_file_path=glove_file_path)

In [143]:
for source in ['from_scratch']: # 'word2vec', 'fasttext',
    print('Source: %s' % (source))
    print(word_embedding.get_vector(source=source, word='cat'))
    print(len(word_embedding.get_vector(source=source, word='cat')))


Source: from_scratch
[ 2.6831627   1.3476357  -1.4496872   0.93928796 -1.9091156  -1.6276546
  2.7061791   3.6508634   0.0967804   0.00791635  1.7608486   1.6253062
 -1.8491086   0.11836294  2.5336206   0.7067682  -0.7633619   0.24489829
 -1.330402    0.24993835  0.3750465  -1.1859053   1.8556513   1.2459805
  2.756841   -1.7531066  -3.2526398  -5.1765423  -0.14817668 -0.18143839
 -0.7891922  -1.7849101  -1.4421382  -2.0194638  -4.299649    4.4633813
  1.4478363   2.3502495   2.810452    1.0866938  -1.2085488   0.03623242
  0.09146767  1.6445264   1.233292    0.20943132 -4.1044407   1.2211795
 -0.38373768 -1.8678396   0.90110326  1.8209499  -1.8441017  -1.1734719
  1.2036536   2.4842527  -1.2873391  -3.553294    1.8199185  -1.587625
  0.8172394  -0.5806897  -1.3481711   0.5547119   2.7400177   0.58666843
  0.30444112 -3.7092197  -2.0056803  -0.5600727  -2.9299028  -0.6160517
 -4.175562   -0.38197666  3.2718177  -0.12175719 -1.941347    0.2476754
 -0.10531218  0.993097    0.00878957 -2.

In [144]:
for source in ['from_scratch']: # 'word2vec', 'fasttext',
    print('Source: %s' % (source))
    print(word_embedding.get_synonym(source=source, word='cat', topn=5))


Source: from_scratch
[('kitty', 0.9006395936012268), ('kitten', 0.8226063251495361), ('dog', 0.8149734735488892), ('feline', 0.7507354617118835), ('friskie', 0.7498754262924194)]


In [146]:
w1 = 'cat'
w2 = 'kitty'

for source in ['from_scratch']: # 'word2vec', 'fasttext'
    print('Source: %s' % (source))
    print(word_embedding.get_distance_between_two_words(source=source,word1=w1, word2=w2))


Source: from_scratch
0.90063965


In [154]:
def tok2vec(tokens, source:str, avg:str):
    vects = word_embedding.get_vectors(source=source, words=tokens)

    if avg == 'mean':
        return np.nanmean(vects, axis=0)
    elif avg == 'sum':
        return np.nansum(vects, axis=0)
    else:
        raise ValueError('Select correct averaging method: sum or mean')


In [155]:
word_embedding = WordEmbedding()

source = 'from_scratch'

word_embedding.load(source=source, file_path=from_scratch_file_path)


2025-12-18 15:42:03.671760 start: loading from_scratch
2025-12-18 15:42:03.706980 end: loading from_scratch


In [157]:
nltk.download('punkt_tab')
train_idxs = df.sample(frac=0.8, random_state=42).index
test_idxs = [idx for idx in df.index if idx not in train_idxs]

X_train = df.loc[train_idxs, 'text_normalized'].apply(
    word_tokenize).apply(lambda x: tok2vec(x, source, 'mean')).to_numpy()

X_test = df.loc[test_idxs, 'text_normalized'].apply(
    word_tokenize).apply(lambda x: tok2vec(x, source, 'mean')).to_numpy()

X_train = np.stack(X_train, axis=0)
X_test = np.stack(X_test, axis=0)

y_train = df.loc[train_idxs, 'sentiment']
y_test = df.loc[test_idxs, 'sentiment']


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Nolan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
C:\Users\Nolan\AppData\Local\Temp\ipykernel_58976\549319055.py:5: RuntimeWarning: Mean of empty slice
  return np.nanmean(vects, axis=0)


In [160]:
# 1. Чистимо дані від пустот
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

# 2. Створюємо модель
model = LogisticRegression(random_state=42, max_iter=1000) # max_iter іноді треба збільшити

# 3. Вчимо
print("Починаю навчання...")
model.fit(X_train, y_train)

# 4. Прогнозуємо
predictions = model.predict(X_test)
# Якщо хочеш точнішу метрику ROC AUC, краще брати ймовірності (predict_proba), а не класи
probs = model.predict_proba(X_test)[:, 1] 

print('AUC Score:', roc_auc_score(y_test, probs))


Починаю навчання...
AUC Score: 0.9306697421302594


In [161]:
text_tokenized = df['text_normalized'].apply(word_tokenize).to_list()

In [162]:
def tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
        
text_tokenized_tagget = list(tagged_document(text_tokenized))

text_tokenized_tagget[3]

TaggedDocument(words=['look', 'secret', 'ingredient', 'robitussin', 'believe', 'find', 'it', 'get', 'addition', 'root', 'beer', 'extract', 'order', 'which', 'good', 'make', 'cherry', 'soda', 'flavor', 'medicinal'], tags=[3])

In [163]:
model_Doc2Vec = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=5, epochs=200)

model_Doc2Vec.build_vocab(text_tokenized_tagget)

model_Doc2Vec.train(text_tokenized_tagget, total_examples=model_Doc2Vec.corpus_count, epochs=model_Doc2Vec.epochs)

In [166]:
# 1. Отримуємо вектори (це ти вже зробив, але на всяк випадок)
X = [model_Doc2Vec.infer_vector(x) for x in text_tokenized]

# 2. !!! МАГІЯ ТУТ !!!
# Перетворюємо список X на Pandas Series і кажемо: 
# "Використовуй ті ж самі номери-індекси, що були в оригінальному DataFrame"
X_series = pd.Series(X, index=df.index)

# 3. Тепер розбиваємо, використовуючи .loc (який шукає по Імені індекса, а не по порядку)
X_train = X_series.loc[train_idxs].tolist()
X_test = X_series.loc[test_idxs].tolist()

# 4. Перетворюємо на масиви Numpy для моделі
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

print("Розміри X_train:", X_train.shape)
print("Розміри X_test:", X_test.shape)

Розміри X_train: (291306, 300)
Розміри X_test: (72827, 300)


In [167]:
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print('AUC: ', roc_auc_score(y_test, predictions))


AUC:  0.7517709698927251
